In [ ]:
import sys

from SVERL_icml_2023.portfolio_DRL.data_function import *
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from torch.utils.data import Dataset, DataLoader
from SVERL_icml_2023.portfolio_DRL.create_model import *
from sklearn.model_selection import TimeSeriesSplit
from SVERL_icml_2023.shapley import Shapley
import shap
sys.path.append("E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023")
# from portfolio_DRL.create_model import StockPredictWrapper
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback
import pandas as pd
import numpy as np
import gymnasium as gym
from gym import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import os

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load data explicitly
shap_metrics = pd.read_csv('shap_value_metrics_export.csv', parse_dates=['Start_Date', 'End_Date'])
factor_returns = pd.read_csv('aligned_factors.csv', parse_dates=['Date'], index_col='Date')
stock_returns = pd.read_csv('daily_returns.csv', parse_dates=['Date'], index_col='Date')

# Explicitly filter only 'Test' phase SHAP metrics
shap_metrics_test = shap_metrics[shap_metrics['Phase'] == 'Test'].copy()

# Verify explicitly
print(f"Total SHAP records (Test Phase): {len(shap_metrics_test)}")

# Select SHAP metrics explicitly
selected_columns = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA',
    'shap_std_Mkt-RF', 'shap_std_SMB', 'shap_std_HML',
    'shap_std_RMW', 'shap_std_CMA',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB', 'mean_abs_over_std_HML',
    'mean_abs_over_std_RMW', 'mean_abs_over_std_CMA'
]

# Aggregate SHAP metrics explicitly by Start_Date and End_Date across stocks
shap_agg = shap_metrics_test.groupby(['Start_Date', 'End_Date'])[selected_columns].mean().reset_index()

# Explicitly scale SHAP metrics
scaler_shap = StandardScaler()
shap_agg[selected_columns] = scaler_shap.fit_transform(shap_agg[selected_columns])

# Verify explicitly scaled SHAP metrics
print("Scaled SHAP metrics (Test phase):", shap_agg.head())

# Explicitly map SHAP intervals to daily returns
daily_regimes = pd.DataFrame(index=stock_returns.index)

for _, row in shap_agg.iterrows():
    mask = (daily_regimes.index >= row['Start_Date']) & (daily_regimes.index <= row['End_Date'])
    daily_regimes.loc[mask, selected_columns] = row[selected_columns].values

# Explicit forward-fill for continuity
daily_regimes.ffill(inplace=True)
daily_regimes.dropna(inplace=True)

# Verify explicitly aligned daily regimes
print("Daily regimes (aligned):", daily_regimes.head())

# Normalize explicitly factor returns
factor_scaler = StandardScaler()
factor_returns_norm = pd.DataFrame(
    factor_scaler.fit_transform(factor_returns),
    index=factor_returns.index,
    columns=factor_returns.columns
)

# Normalize explicitly stock returns
stock_scaler = StandardScaler()
stock_returns_norm = pd.DataFrame(
    stock_scaler.fit_transform(stock_returns),
    index=stock_returns.index,
    columns=stock_returns.columns
)

# Merge explicitly into final dataset
merged_data = stock_returns_norm.join(daily_regimes, how='inner').join(factor_returns_norm, rsuffix='_factor').dropna()

# Explicit final dataset verification
print("Merged normalized data preview:", merged_data.head())

# Save explicitly final aligned dataset
merged_data.to_csv("final_merged_data.csv")

start_date = merged_data.index.min()
end_date = merged_data.index.max()

# Explicitly filter raw stock returns to match the merged_data dates
raw_stock_returns = stock_returns.loc[start_date:end_date].copy()
raw_stock_returns.to_csv("aligned_raw_stock_returns.csv")

stock_returns_norm_filter = stock_returns_norm.loc[start_date:end_date].copy()

In [ ]:
import gym
from gym import spaces
import numpy as np

class SinglePPOEnv(gym.Env):
    def __init__(self, returns_df, regime_df, raw_returns_df):
        super().__init__()
        self.stock_returns = returns_df.values
        self.regime_features = regime_df.values
        self.raw_stock_returns = raw_returns_df.values
        self.current_step = 0
        self.n_assets = self.stock_returns.shape[1]

        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)

        # Adjust observation space shape accordingly
        obs_dim = self.n_assets + self.regime_features.shape[1]
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self._get_obs()

    def _get_obs(self):
        stock_returns = self.stock_returns[self.current_step]
        regime = self.regime_features[self.current_step]

        obs = np.concatenate([stock_returns, regime])
        return np.nan_to_num(obs)

    def step(self, action):
        action_sum = action.sum()
        action = action / action_sum if action_sum != 0 else np.ones_like(action) / len(action)

        # Reward calculated explicitly using raw returns
        reward = np.dot(action, self.raw_stock_returns[self.current_step + 1])

        self.current_step += 1
        done = self.current_step >= len(self.stock_returns) - 1
        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)
        reward = 0.0 if np.isnan(reward) else reward
        return obs, reward, done, {}


In [ ]:
# reb every 21 days
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_single_ppo/'

train_window, test_window = 252, 63
rebalance_freq = 21  # rebalance every 21 days
tickers = stock_returns_norm.columns.tolist()
selected_columns = daily_regimes.columns.tolist()
factor_columns = factor_returns_norm.columns.tolist()
use_factor_returns = False
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = False

total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Training Window {i+1}/{total_windows}")
    print(f"Training Period: {merged_data.index[start].date()} to {merged_data.index[start + train_window - 1].date()}")

    # Explicit training and testing dataset slicing
    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    if train_df.isna().values.any() or test_df.isna().values.any():
        print(f"Skipping window {i+1} due to NaNs.")
        continue

    # Train PPO model explicitly
    model_filename = os.path.join(model_save_dir, f"ppo_model_window_{i}.zip")

    # Setup environment
    env = DummyVecEnv([
        lambda: SinglePPOEnv(
            train_df[tickers],
            train_df[selected_columns + factor_columns] if use_factor_returns else train_df[selected_columns],
            raw_returns_train
        )
    ])


     # Load existing model if indicated and file exists
    if load_existing_model and i > 0 and os.path.exists(os.path.join(model_save_dir, f"ppo_model_window_{i-1}.zip")):
        previous_model_filename = os.path.join(model_save_dir, f"ppo_model_window_{i-1}.zip")
        print(f"Loading and retraining previous PPO model from {previous_model_filename}")
        model = PPO.load(previous_model_filename, env=env)
        model.learn(total_timesteps=10000)
    else:
        print("Training new PPO model from scratch...")
        model = PPO('MlpPolicy', env, verbose=0)
        model.learn(total_timesteps=10000)

    model.save(model_filename)
    print(f"Model saved to {model_filename}")

    # Testing phase setup
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[selected_columns + factor_columns].values if use_factor_returns else test_df[selected_columns].values

    print(f"\nTesting Period: {test_dates[0].date()} to {test_dates[-1].date()} (63 days total)")

    # Evaluate every 21 days (3 rebalances per testing window)
    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        print(f"\n--- Rebalance {rebalance_num + 1} ---")
        print(f"Rebalance date: {test_dates[rebalance_start].date()}")
        print(f"Holding period: {test_dates[rebalance_start].date()} to {test_dates[rebalance_end - 1].date()}")

        obs_returns = test_returns[rebalance_start:rebalance_end]
        obs_regimes = test_regimes[rebalance_start:rebalance_end]
        obs_raw_returns = raw_returns_test.iloc[rebalance_start:rebalance_end]

        env_test = SinglePPOEnv(
            returns_df=pd.DataFrame(obs_returns, columns=tickers),
            regime_df=pd.DataFrame(obs_regimes, columns=(selected_columns + factor_columns) if use_factor_returns else selected_columns),
            raw_returns_df=obs_raw_returns
        )

        obs = env_test.reset()
        action, _ = model.predict(obs, deterministic=True)

        # Explicit normalization right here:
        action_sum = action.sum()
        if action_sum != 0:
            action /= action_sum
        else:
            action = np.ones_like(action) / len(action)

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        print(f"Normalized Portfolio weights:\n{weights_dict}")

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, obs_raw_returns.iloc[step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward for holding period: {cumulative_reward:.4f}")

print("\n" + "="*30 + "\nAll training/testing windows completed successfully.\n" + "="*30)

# Save final results explicitly
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('single_step_allocations_dates_rebalanced.csv', index=False)

print("Final results saved to 'single_step_allocations_dates_rebalanced.csv'")


In [ ]:
print("\n" + "="*30 + "\nAll training/testing windows completed successfully.\n" + "="*30)

# Save final results explicitly
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('single_step_allocations_dates_rebalanced.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
# Load data explicitly
shap_metrics = pd.read_csv('shap_value_metrics_export.csv', parse_dates=['Start_Date', 'End_Date'])
factor_returns = pd.read_csv('aligned_factors.csv', parse_dates=['Date'], index_col='Date')
stock_returns = pd.read_csv('daily_returns.csv', parse_dates=['Date'], index_col='Date')

# Explicitly filter only 'Test' phase SHAP metrics
shap_metrics_test = shap_metrics[shap_metrics['Phase'] == 'Test'].copy()

# Verify explicitly
print(f"Total SHAP records (Test Phase): {len(shap_metrics_test)}")

# Select SHAP metrics explicitly
selected_columns = [
    'mean_abs_shap_Mkt-RF', 'mean_abs_shap_SMB', 'mean_abs_shap_HML',
    'mean_abs_shap_RMW', 'mean_abs_shap_CMA',
    'shap_std_Mkt-RF', 'shap_std_SMB', 'shap_std_HML',
    'shap_std_RMW', 'shap_std_CMA',
    'mean_abs_over_std_Mkt-RF', 'mean_abs_over_std_SMB', 'mean_abs_over_std_HML',
    'mean_abs_over_std_RMW', 'mean_abs_over_std_CMA'
]

# Aggregate SHAP metrics explicitly by Start_Date and End_Date across stocks
shap_agg = shap_metrics_test.groupby(['Start_Date', 'End_Date'])[selected_columns].mean().reset_index()

# Explicitly scale SHAP metrics
scaler_shap = StandardScaler()
shap_agg[selected_columns] = scaler_shap.fit_transform(shap_agg[selected_columns])

# Verify explicitly scaled SHAP metrics
# print("Scaled SHAP metrics (Test phase):", shap_agg.head())

# Explicitly map SHAP intervals to daily returns
daily_regimes = pd.DataFrame(index=stock_returns.index)

for _, row in shap_agg.iterrows():
    mask = (daily_regimes.index >= row['Start_Date']) & (daily_regimes.index <= row['End_Date'])
    daily_regimes.loc[mask, selected_columns] = row[selected_columns].values

# Explicit forward-fill for continuity
daily_regimes.ffill(inplace=True)
daily_regimes.dropna(inplace=True)

# Perform KMeans clustering explicitly on daily regimes
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
daily_regimes['Cluster'] = kmeans.fit_predict(daily_regimes[selected_columns])
# daily_regimes['Cluster'] = 1 - daily_regimes['Cluster']
# cluster_mapping = {0: 1, 1: 0}  # Good explicitly set as 1
# daily_regimes['Cluster_Mapped'] = daily_regimes['Cluster'].map(cluster_mapping)

from sklearn.cluster import KMeans

# Example: Clustering raw returns into regimes
# kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
# regime_labels = kmeans.fit_predict(raw_future_returns_df)
# regime_labels_series = pd.Series(regime_labels, index=raw_future_returns_df.index)

# Verify explicitly aligned daily regimes
# print("Daily regimes (aligned):", daily_regimes.head())

# Normalize explicitly factor returns
factor_scaler = StandardScaler()
factor_returns_norm = pd.DataFrame(
    factor_scaler.fit_transform(factor_returns),
    index=factor_returns.index,
    columns=factor_returns.columns
)

# Normalize explicitly stock returns
stock_scaler = StandardScaler()
stock_returns_norm = pd.DataFrame(
    stock_scaler.fit_transform(stock_returns),
    index=stock_returns.index,
    columns=stock_returns.columns
)

# Merge explicitly into final dataset
merged_data = stock_returns_norm.join(daily_regimes[['Cluster'] + selected_columns], how='inner')\
                                .join(factor_returns_norm, rsuffix='_factor')\
                                .dropna()

# Explicit final dataset verification
# print("Merged normalized data preview:", merged_data.head())

# Save explicitly final aligned dataset
merged_data.to_csv("final_merged_data.csv")
# Explicitly filter raw stock returns to match the merged_data dates
start_date = merged_data.index.min()
end_date = merged_data.index.max()

# Explicitly filter raw stock returns to match the merged_data dates
raw_stock_returns = stock_returns.loc[start_date:end_date].copy()
raw_stock_returns.to_csv("aligned_raw_stock_returns.csv")

stock_returns_norm_filter = stock_returns_norm.loc[start_date:end_date].copy()
cluster_avg_returns = raw_stock_returns.join(daily_regimes['Cluster']).groupby('Cluster').mean().mean(axis=1)
print("Explicit Cluster Mean Returns Check:\n", cluster_avg_returns)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt

# Use existing explicitly scaled SHAP metrics from daily_regimes
regime_data = daily_regimes.values
cluster_range = range(2, 11)  # Evaluate 2 to 10 clusters explicitly
inertia = []
silhouette_scores = []
calinski_scores = []

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(regime_data)

    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(regime_data, labels))
    calinski_scores.append(calinski_harabasz_score(regime_data, labels))

    print(f"Clusters: {k} | Inertia: {kmeans.inertia_:.2f} | "
          f"Silhouette: {silhouette_scores[-1]:.4f} | "
          f"Calinski-Harabasz: {calinski_scores[-1]:.2f}")
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

ax[0].plot(cluster_range, inertia, 'bo-', linewidth=2)
ax[0].set_title('Elbow Method (Inertia)', fontsize=14)
ax[0].set_xlabel('Number of Clusters', fontsize=12)
ax[0].set_ylabel('Inertia', fontsize=12)

ax[1].plot(cluster_range, silhouette_scores, 'go-', linewidth=2)
ax[1].set_title('Silhouette Score', fontsize=14)
ax[1].set_xlabel('Number of Clusters', fontsize=12)
ax[1].set_ylabel('Silhouette Score', fontsize=12)

ax[2].plot(cluster_range, calinski_scores, 'ro-', linewidth=2)
ax[2].set_title('Calinski-Harabasz Index', fontsize=14)
ax[2].set_xlabel('Number of Clusters', fontsize=12)
ax[2].set_ylabel('Calinski-Harabasz Score', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
import gym
import numpy as np
from gym import spaces

class RegimeSelectorEnv(gym.Env):
    def __init__(self, regime_features_df, raw_future_returns_df):
        super().__init__()
        self.regime_features = regime_features_df.values
        # self.raw_future_returns = raw_future_returns_df.values
        self.raw_future_returns = raw_future_returns_df
        
        self.current_step = 0
        self.n_regimes = 2  # Regimes: 0 = bad, 1 = good

        self.action_space = spaces.Discrete(self.n_regimes)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, 
            shape=(self.regime_features.shape[1],),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self.regime_features[self.current_step]

    def step(self, action):
        avg_future_return = np.mean(self.raw_future_returns[self.current_step])

        # Reward logic explicitly explained:
        # - action = 1 (predict good regime): reward = avg_future_return
        # - action = 0 (predict bad regime): reward = -avg_future_return
        # Thus, correct predictions yield positive rewards, incorrect yield negative rewards.
        reward = avg_future_return if action == 1 else -avg_future_return

        self.current_step += 1
        done = self.current_step >= len(self.regime_features) - 1

        obs = (
            self.regime_features[self.current_step]
            if not done else np.zeros(self.observation_space.shape)
        )

        return obs, reward, done, {}



import gym
import numpy as np
from gym import spaces

class RegimeConditionedPortfolioEnv(gym.Env):
    def __init__(self, returns_df, chosen_regime_vector, raw_returns_df, factor_returns_df=None):
        super().__init__()
        
        self.stock_returns = returns_df.values
        self.chosen_regime_vector = chosen_regime_vector
        self.raw_stock_returns = raw_returns_df.values
        self.factor_returns = factor_returns_df.values if factor_returns_df is not None else None
        
        self.current_step = 0
        self.n_assets = self.stock_returns.shape[1]

        self.action_space = spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        
        obs_dim = self.n_assets + 1
        if self.factor_returns is not None:
            obs_dim += self.factor_returns.shape[1]

        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(obs_dim,),
            dtype=np.float32
        )

    def reset(self):
        self.current_step = 0
        return self._get_obs()

    def _get_obs(self):
        returns = self.stock_returns[self.current_step]
        regime = np.array([self.chosen_regime_vector[self.current_step]])
        obs = np.concatenate([returns, regime])

        if self.factor_returns is not None:
            factor_obs = self.factor_returns[self.current_step]
            obs = np.concatenate([obs, factor_obs])

        return np.nan_to_num(obs)

    def step(self, action):
        action_sum = action.sum()
        if action_sum == 0:
            action = np.ones_like(action) / len(action)
        else:
            action = action / action_sum

        # Intuitive reward: explicitly realized portfolio return
        reward = np.dot(action, self.raw_stock_returns[self.current_step + 1])

        self.current_step += 1
        done = self.current_step >= len(self.stock_returns) - 1

        obs = self._get_obs() if not done else np.zeros(self.observation_space.shape)

        reward = 0.0 if np.isnan(reward) else reward

        return obs, reward, done, {}


In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import os
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/'

train_window, test_window = 252, 63
rebalance_freq = 21
use_factor_returns = False  # Set this flag as needed
tickers = stock_returns_norm.columns.tolist()
# regime_columns = daily_regimes.columns.tolist() #### remove cluster in the feature set!!
regime_columns = daily_regimes.columns.drop('Cluster').tolist()
# regime_columns = daily_regimes[['Cluster']].columns.tolist()
factor_columns = factor_returns_norm.columns.tolist() if use_factor_returns else []
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = False
total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Hierarchical Training Window {i+1}/{total_windows}")
    high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")

    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    if train_df.isna().values.any() or test_df.isna().values.any():
        print(f"Skipping window {i+1} due to NaNs.")
        continue

    # High-Level training data preparation (no look-ahead bias)
    regime_features_high = train_df[regime_columns + factor_columns][:-rebalance_freq]
    future_returns_high = np.array([
        raw_returns_train.iloc[idx + 1: idx + rebalance_freq + 1].mean().mean()
        for idx in range(len(regime_features_high))
    ])

    high_env = DummyVecEnv([
        lambda: RegimeSelectorEnv(regime_features_high, future_returns_high)
    ])
    
    
    # Load existing high-level model or train a new one
    if load_existing_model and os.path.exists(high_model_path):
        print("Loading existing High-Level Regime Selector model...")
        high_model = PPO.load(high_model_path, env=high_env)
        high_model.learn(total_timesteps=3000)  # continue training briefly
        print(f"Continued training on High-Level model. Saved again to {high_model_path}.")
    else:
        print("Training new High-Level Regime Selector model...")
        # policy_kwargs = dict(net_arch=[128, 64], activation_fn=nn.Tanh)
        # 
        # high_model = PPO(
        #     'MlpPolicy', high_env, verbose=0,
        #     learning_rate=1e-4,
        #     n_steps=2048,
        #     batch_size=128,
        #     gamma=0.95,
        #     ent_coef=0.005,  # Slightly lower entropy encourages differentiation
        #     clip_range=0.2,
        #     n_epochs=10,
        #     policy_kwargs=policy_kwargs
        # )
        # high_model.learn(total_timesteps=10000)
    
        print("Training High-Level Regime Selector...")
        high_model = PPO('MlpPolicy', high_env, verbose=0)
        high_model.learn(total_timesteps=10000)
    # high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")
    # high_model.save(high_model_path)
    # print(f"High-level model saved at: {high_model_path}")

    # Generate historical regimes for low-level model training (no look-ahead)
    historical_regimes = []
    for idx in range(len(regime_features_high)):
        obs = regime_features_high.iloc[idx].values
        regime, _ = high_model.predict(obs, deterministic=True)
        historical_regimes.append(int(regime))

    historical_regimes = pd.Series(historical_regimes, index=regime_features_high.index)

    # Train low-level models per historical regime
    low_level_models = {}
    for regime in historical_regimes.unique():
        low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        regime_indices = historical_regimes[historical_regimes == regime].index
        regime_df = train_df.loc[regime_indices]
        chosen_regime_vector_train = np.repeat(regime, len(regime_df))

        low_env_train = DummyVecEnv([
            lambda: RegimeConditionedPortfolioEnv(
                returns_df=regime_df[tickers],
                chosen_regime_vector=chosen_regime_vector_train,
                raw_returns_df=raw_returns_train.loc[regime_df.index]
            )
        ])
        
        if load_existing_model and os.path.exists(low_model_path):
            print(f"Loading existing Low-Level model for regime {regime}...")
            low_model = PPO.load(low_model_path, env=low_env_train)
            low_model.learn(total_timesteps=3000)  # brief continued training
            print(f"Continued training on Low-Level model regime {regime}. Saved again to {low_model_path}.")
        else:
            print(f"Training new Low-Level model for regime {regime}...")
            # low_model = PPO(
            #     'MlpPolicy', low_env_train, verbose=0,
            #     learning_rate=5e-5,
            #     n_steps=2048,
            #     batch_size=128,
            #     gamma=0.97,
            #     ent_coef=0.003,  # Reduced entropy to discourage uniform policy
            #     clip_range=0.2,
            #     n_epochs=10,
            #     policy_kwargs=policy_kwargs
            # )
            # low_model.learn(total_timesteps=10000)

            low_model = PPO('MlpPolicy', low_env_train, verbose=0)
            low_model.learn(total_timesteps=10000)
        # low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        # low_model.save(low_model_path)
        low_level_models[regime] = low_model
        # print(f"Low-level model for regime {regime} saved at: {low_model_path}")

    # Testing
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[regime_columns + factor_columns].values

    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        high_obs = test_regimes[rebalance_start]
        chosen_regime, _ = high_model.predict(high_obs, deterministic=True)
        chosen_regime = int(chosen_regime.item())
        # high_predictions = [high_model.predict(high_obs, deterministic=False)[0].item() for _ in range(200)]
        # chosen_regime = int(pd.Series(high_predictions).mode()[0])  # take most common predicted regime
        
        chosen_regime_vector_test = np.repeat(chosen_regime, rebalance_end - rebalance_start)
        print(f'chosen_regime = {chosen_regime} and check is {chosen_regime in low_level_models}')
        if chosen_regime in low_level_models:
            low_model = low_level_models[chosen_regime]
            low_env_test = RegimeConditionedPortfolioEnv(
                returns_df=pd.DataFrame(test_returns[rebalance_start:rebalance_end], columns=tickers),
                chosen_regime_vector=chosen_regime_vector_test,
                raw_returns_df=raw_returns_test.iloc[rebalance_start:rebalance_end]
            )

            low_obs = low_env_test.reset()
            action, _ = low_model.predict(low_obs, deterministic=True)
            action /= action.sum()
            # action_samples = np.array([
            #     low_model.predict(low_obs, deterministic=False)[0]
            #     for _ in range(200)
            # ])
            # action_mean = action_samples.mean(axis=0)
            # action_mean /= action_mean.sum()  # normalization
            # action = action_mean
            
        else:
            action = np.ones(len(tickers)) / len(tickers)

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, raw_returns_test.iloc[rebalance_start + step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Chosen_Regime': chosen_regime,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward: {cumulative_reward:.4f}")

result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")


In [ ]:
############################
# retrain model using the previous trained agents for high and low:
############################

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import os
import pandas as pd
import numpy as np

model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/'

train_window, test_window = 252, 42
rebalance_freq = 42
use_factor_returns = False  # Set this flag as needed
tickers = stock_returns_norm.columns.tolist()
regime_columns = daily_regimes.columns.tolist() 
# regime_columns = daily_regimes.columns.drop('Cluster').tolist() #### remove cluster in the feature set!!
# regime_columns = daily_regimes[['Cluster']].columns.tolist()
factor_columns = factor_returns_norm.columns.tolist() if use_factor_returns else []
records = []
raw_stock_returns = stock_returns_norm_filter
load_existing_model = False
total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
    print(f"\n{'='*30}\nStarting Hierarchical Training Window {i+1}/{total_windows}")
    
    # Model paths explicitly defined for high and low-level models
    high_model_path = os.path.join(model_save_dir, f"high_model_window_{i-1}.zip")  # Load previous model
    new_high_model_path = os.path.join(model_save_dir, f"high_model_window_{i}.zip")  # Save current model
    
    # Data splitting
    train_df = merged_data.iloc[start:start + train_window]
    test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
    raw_returns_train = raw_stock_returns.loc[train_df.index]
    raw_returns_test = raw_stock_returns.loc[test_df.index]

    # High-Level training data preparation (no look-ahead bias)
    regime_features_high = train_df[regime_columns + factor_columns][:-rebalance_freq]
    future_returns_high = np.array([
        raw_returns_train.iloc[idx + 1: idx + rebalance_freq + 1].mean().mean()
        for idx in range(len(regime_features_high))
    ])

    high_env = DummyVecEnv([
        lambda: RegimeSelectorEnv(regime_features_high, future_returns_high)
    ])
    
    # Load and continue training from previous window's model
    if load_existing_model and i > 0 and os.path.exists(high_model_path):
        print("Loading existing High-Level model from previous window...")
        high_model = PPO.load(high_model_path, env=high_env)
        high_model.learn(total_timesteps=20000)  # shorter fine-tuning
    else:
        print("Training new High-Level Regime Selector from scratch...")
        high_model = PPO('MlpPolicy', high_env, verbose=0)
        high_model.learn(total_timesteps=50000)  # full training only for first window
    
    # Save updated high-level model explicitly for next window's reuse
    high_model.save(new_high_model_path)
    print(f"High-level model saved at: {new_high_model_path}")

    # Generate historical regimes using the trained high-level model
    historical_regimes = []
    for idx in range(len(regime_features_high)):
        obs = regime_features_high.iloc[idx].values
        regime, _ = high_model.predict(obs, deterministic=True)
        historical_regimes.append(int(regime))

    historical_regimes = pd.Series(historical_regimes, index=regime_features_high.index)

    # Low-level model training with incremental training
    low_level_models = {}
    for regime in historical_regimes.unique():
        prev_low_model_path = os.path.join(model_save_dir, f"low_model_window_{i-1}_regime_{regime}.zip")
        current_low_model_path = os.path.join(model_save_dir, f"low_model_window_{i}_regime_{regime}.zip")
        
        regime_indices = historical_regimes[historical_regimes == regime].index
        regime_df = train_df.loc[regime_indices]
        chosen_regime_vector_train = np.repeat(regime, len(regime_df))

        low_env_train = DummyVecEnv([
            lambda: RegimeConditionedPortfolioEnv(
                returns_df=regime_df[tickers],
                chosen_regime_vector=chosen_regime_vector_train,
                raw_returns_df=raw_returns_train.loc[regime_df.index]
            )
        ])
        
        if load_existing_model and i > 0 and os.path.exists(prev_low_model_path):
            print(f"Loading existing Low-Level model for regime {regime} from previous window...")
            low_model = PPO.load(prev_low_model_path, env=low_env_train)
            low_model.learn(total_timesteps=20000)  # shorter fine-tuning
        else:
            print(f"Training new Low-Level model for regime {regime} from scratch...")
            low_model = PPO('MlpPolicy', low_env_train, verbose=0)
            low_model.learn(total_timesteps=10000)  # full training for first window or new regime

        low_model.save(current_low_model_path)
        low_level_models[regime] = low_model
        print(f"Low-level model for regime {regime} saved at: {current_low_model_path}")


    # Testing
    test_dates = test_df.index
    test_returns = test_df[tickers].values
    test_regimes = test_df[regime_columns + factor_columns].values

    for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
        rebalance_end = min(rebalance_start + rebalance_freq, test_window)

        high_obs = test_regimes[rebalance_start]
        chosen_regime, _ = high_model.predict(high_obs, deterministic=True)
        chosen_regime = int(chosen_regime.item())
        # high_predictions = [high_model.predict(high_obs, deterministic=False)[0].item() for _ in range(200)]
        # chosen_regime = int(pd.Series(high_predictions).mode()[0])  # take most common predicted regime
        
        chosen_regime_vector_test = np.repeat(chosen_regime, rebalance_end - rebalance_start)
        print(f'chosen_regime = {chosen_regime} and check is {chosen_regime in low_level_models}')
        if chosen_regime in low_level_models:
            low_model = low_level_models[chosen_regime]
            low_env_test = RegimeConditionedPortfolioEnv(
                returns_df=pd.DataFrame(test_returns[rebalance_start:rebalance_end], columns=tickers),
                chosen_regime_vector=chosen_regime_vector_test,
                raw_returns_df=raw_returns_test.iloc[rebalance_start:rebalance_end]
            )

            low_obs = low_env_test.reset()
            action, _ = low_model.predict(low_obs, deterministic=True)
            action /= action.sum()
            # action_samples = np.array([
            #     low_model.predict(low_obs, deterministic=False)[0]
            #     for _ in range(200)
            # ])
            # action_mean = action_samples.mean(axis=0)
            # action_mean /= action_mean.sum()  # normalization
            # action = action_mean
            
        else:
            action = np.ones(len(tickers)) / len(tickers)

        portfolio_value = 1.0
        for step in range(rebalance_end - rebalance_start - 1):
            reward = np.dot(action, raw_returns_test.iloc[rebalance_start + step + 1])
            portfolio_value *= (1 + reward)
        cumulative_reward = portfolio_value - 1.0

        weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
        records.append({
            'Rebalance_Date': test_dates[rebalance_start],
            'Window': i,
            'Rebalance_Number': rebalance_num,
            'Chosen_Regime': chosen_regime,
            'Holding_Period_Start': test_dates[rebalance_start],
            'Holding_Period_End': test_dates[rebalance_end - 1],
            'Cumulative_Reward': cumulative_reward,
            **weights_dict
        })

        print(f"Cumulative reward: {cumulative_reward:.4f}")

result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")



In [ ]:
result_df = pd.DataFrame(records)
result_df['Cumulative_Return'] = result_df['Cumulative_Reward'].cumsum()
result_df.to_csv('hierarchical_allocations_dates_rebalanced.csv', index=False)

print("\nHierarchical training/testing completed successfully.")
print("Final hierarchical results saved to 'hierarchical_allocations_dates_rebalanced.csv'")


In [ ]:
##### parallel running
##### Parallel Running with Progress Indicators
import os
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Set your directories and parameters explicitly
model_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/trained_hierarchical_model/'
results_save_dir = 'E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/results/'

os.makedirs(model_save_dir, exist_ok=True)
os.makedirs(results_save_dir, exist_ok=True)

# Your configurations
train_window, test_window = 252, 63
rebalance_freq = 21
use_factor_returns = False

tickers = stock_returns_norm.columns.tolist()
regime_columns = daily_regimes.columns.tolist()
factor_columns = factor_returns_norm.columns.tolist() if use_factor_returns else []

total_runs = 25
load_existing_model = False

final_records = []  # To collect all results across 100 runs

# Main sequential loop
for run_number in range(total_runs):
    print(f"\n{'='*60}\n🚀 Starting Run {run_number + 1}/{total_runs}\n{'='*60}")
    run_records = []

    total_windows = len(range(0, len(merged_data) - train_window - test_window, test_window))

    for i, start in enumerate(range(0, len(merged_data) - train_window - test_window, test_window)):
        # print(f"\n🔸 Run {run_number + 1}, Training Window {i + 1}/{total_windows}")

        # Model path definitions
        high_model_path_prev = os.path.join(model_save_dir, f"run{run_number}_high_model_window_{i-1}.zip")
        high_model_path_curr = os.path.join(model_save_dir, f"run{run_number}_high_model_window_{i}.zip")

        # Data preparation
        train_df = merged_data.iloc[start:start + train_window]
        test_df = merged_data.iloc[start + train_window:start + train_window + test_window + 1]
        raw_returns_train = stock_returns_norm_filter.loc[train_df.index]
        raw_returns_test = stock_returns_norm_filter.loc[test_df.index]

        # High-level regime features
        regime_features_high = train_df[regime_columns + factor_columns][:-rebalance_freq]
        future_returns_high = np.array([
            raw_returns_train.iloc[idx + 1: idx + rebalance_freq + 1].mean().mean()
            for idx in range(len(regime_features_high))
        ])

        high_env = DummyVecEnv([
            lambda: RegimeSelectorEnv(regime_features_high, future_returns_high)
        ])

        # High-level model training
        if load_existing_model and i > 0 and os.path.exists(high_model_path_prev):
            # print(f"🔄 Loading and retraining High-Level model from previous window {i-1}...")
            high_model = PPO.load(high_model_path_prev, env=high_env)
            high_model.learn(total_timesteps=20000)
        else:
            # print("🆕 Training new High-Level Regime Selector model from scratch...")
            high_model = PPO('MlpPolicy', high_env, verbose=0)
            high_model.learn(total_timesteps=10000)

        high_model.save(high_model_path_curr)

        # Generate historical regimes
        historical_regimes = [
            int(high_model.predict(regime_features_high.iloc[idx].values, deterministic=True)[0])
            for idx in range(len(regime_features_high))
        ]
        historical_regimes_series = pd.Series(historical_regimes, index=regime_features_high.index)

        # Low-level model training per regime
        low_level_models = {}
        for regime in historical_regimes_series.unique():
            low_model_path_prev = os.path.join(model_save_dir, f"run{run_number}_low_model_window_{i-1}_regime_{regime}.zip")
            low_model_path_curr = os.path.join(model_save_dir, f"run{run_number}_low_model_window_{i}_regime_{regime}.zip")

            regime_indices = historical_regimes_series[historical_regimes_series == regime].index
            regime_df = train_df.loc[regime_indices]
            chosen_regime_vector_train = np.repeat(regime, len(regime_df))

            low_env_train = DummyVecEnv([
                lambda: RegimeConditionedPortfolioEnv(
                    returns_df=regime_df[tickers],
                    chosen_regime_vector=chosen_regime_vector_train,
                    raw_returns_df=raw_returns_train.loc[regime_df.index]
                )
            ])

            if load_existing_model and i > 0 and os.path.exists(low_model_path_prev):
                # print(f"🔄 Loading and retraining Low-Level model for regime {regime} from previous window...")
                low_model = PPO.load(low_model_path_prev, env=low_env_train)
                low_model.learn(total_timesteps=20000)
            else:
                # print(f"🆕 Training new Low-Level model for regime {regime} from scratch...")
                low_model = PPO('MlpPolicy', low_env_train, verbose=0)
                low_model.learn(total_timesteps=10000)

            low_model.save(low_model_path_curr)
            low_level_models[regime] = low_model

        # Testing
        for rebalance_num, rebalance_start in enumerate(range(0, test_window, rebalance_freq)):
            rebalance_end = min(rebalance_start + rebalance_freq, test_window)
            high_obs = test_df[regime_columns + factor_columns].iloc[rebalance_start].values
            chosen_regime = int(high_model.predict(high_obs, deterministic=True)[0])

            chosen_regime_vector_test = np.repeat(chosen_regime, rebalance_end - rebalance_start)

            if chosen_regime in low_level_models:
                low_model = low_level_models[chosen_regime]
                low_env_test = RegimeConditionedPortfolioEnv(
                    returns_df=test_df[tickers].iloc[rebalance_start:rebalance_end],
                    chosen_regime_vector=chosen_regime_vector_test,
                    raw_returns_df=raw_returns_test.iloc[rebalance_start:rebalance_end]
                )
                low_obs = low_env_test.reset()
                action, _ = low_model.predict(low_obs, deterministic=True)
                action /= action.sum()
            else:
                action = np.ones(len(tickers)) / len(tickers)

            portfolio_value = 1.0
            for step in range(rebalance_end - rebalance_start - 1):
                reward = np.dot(action, raw_returns_test.iloc[rebalance_start + step + 1])
                portfolio_value *= (1 + reward)
            cumulative_reward = portfolio_value - 1.0

            weights_dict = {ticker: round(weight, 4) for ticker, weight in zip(tickers, action)}
            run_records.append({
                'Run': run_number,
                'Rebalance_Date': test_df.index[rebalance_start],
                'Window': i,
                'Rebalance_Number': rebalance_num,
                'Chosen_Regime': chosen_regime,
                'Cumulative_Reward': cumulative_reward,
                **weights_dict
            })

            print(f"✔️ Run {run_number + 1}, Window {i + 1}, Rebalance {rebalance_num + 1}: Cumulative reward: {cumulative_reward:.4f}")

    # Save individual run result
    run_df = pd.DataFrame(run_records)
    run_df.to_csv(f'{results_save_dir}hierarchical_run_{run_number}.csv', index=False)
    final_records.append(run_df)

# Combine all results after all runs
final_df = pd.concat(final_records, ignore_index=True)
final_df.to_csv(f'{results_save_dir}consolidated_hierarchical_results.csv', index=False)

print("\n🎉 All 25 runs completed successfully!")
print(f"✅ Consolidated results saved to '{results_save_dir}consolidated_hierarchical_results.csv'")


LOAD and CALC Portfolio Return

In [16]:
import pandas as pd
import numpy as np

# Load data explicitly
port_wts = pd.read_csv('port_wt_test_old.csv', parse_dates=['Date'], index_col='Date')
daily_returns = pd.read_csv('daily_returns_10ETFs.csv', parse_dates=['Date'], index_col='Date')

common_tickers = [col for col in port_wts.columns if col in daily_returns.columns]
daily_returns = daily_returns[common_tickers]

# Explicitly filter daily returns to the date range covered by portfolio weights
start_date, end_date = port_wts.index.min(), port_wts.index.max() + pd.Timedelta(days=21)
daily_returns = daily_returns.loc[start_date:end_date]

# Initialize drifted weights with the first available rebalance weights
initial_weights = port_wts.loc[start_date].values

equal_weight = np.array([1.0 / len(common_tickers)] * len(common_tickers))

# Create drifted weights DataFrame explicitly initialized
drifted_weights = pd.DataFrame(index=daily_returns.index, columns=common_tickers)
equal_weights = pd.DataFrame(index=daily_returns.index, columns=common_tickers)

current_weights = initial_weights
current_equal_weights = equal_weight

# Initialize returns DataFrame explicitly
returns_df = pd.DataFrame(index=daily_returns.index, columns=['Optimal_Portfolio_Return', 'Equal_Weight_Return'])

for current_date in daily_returns.index:
    if current_date in port_wts.index:
        # Explicit rebalance date: assign new weights
        current_weights = port_wts.loc[current_date].values
        current_equal_weights = equal_weight
    else:
        # Explicitly drift weights using previous day's return
        prev_day_return = daily_returns.loc[current_date]

        drifted_wts_numerator = current_weights * (1 + prev_day_return.values)
        current_weights = drifted_wts_numerator / np.sum(drifted_wts_numerator)

        equal_drifted_numerator = current_equal_weights * (1 + prev_day_return.values)
        current_equal_weights = equal_drifted_numerator / np.sum(equal_drifted_numerator)

    drifted_weights.loc[current_date] = current_weights
    equal_weights.loc[current_date] = current_equal_weights
    shifted_drifted_weights = drifted_weights.shift(1)
    shifted_equal_weights = equal_weights.shift(1)
    if current_date == daily_returns.index[0]:
        # On the first day, use initial weights directly
        returns_df.loc[current_date, 'Optimal_Portfolio_Return'] = np.dot(
            drifted_weights.loc[current_date], daily_returns.loc[current_date])
        returns_df.loc[current_date, 'Equal_Weight_Return'] = np.dot(
            equal_weights.loc[current_date], daily_returns.loc[current_date])
    else:
        # Explicitly use previous day's weights
        returns_df.loc[current_date, 'Optimal_Portfolio_Return'] = np.dot(
            shifted_drifted_weights.loc[current_date], daily_returns.loc[current_date])
        returns_df.loc[current_date, 'Equal_Weight_Return'] = np.dot(
            shifted_equal_weights.loc[current_date], daily_returns.loc[current_date])

# Check explicitly
print("Drifted weights (head):\n", drifted_weights.head())
print("\nPortfolio returns (head):\n", returns_df.head())

# Save explicitly
drifted_weights.to_csv('drifted_weights_corrected.csv')
equal_weights.to_csv('equal_weights.csv')
returns_df.to_csv('portfolio_returns_combined.csv')



Drifted weights (head):
                  XLB  XLE       XLF       XLI  XLK  XLP       XLY       XLV  \
Date                                                                          
2017-03-14  0.277734  0.0  0.005614  0.279801  0.0  0.0  0.239181  0.039133   
2017-03-15   0.27886  0.0  0.005539  0.279645  0.0  0.0  0.237612  0.039158   
2017-03-16  0.278435  0.0  0.005578  0.279862  0.0  0.0  0.238857  0.038987   
2017-03-17  0.278897  0.0  0.005507  0.280261  0.0  0.0  0.238569  0.038652   
2017-03-20  0.280636  0.0  0.005465  0.279942  0.0  0.0  0.238161  0.038622   

                 XLU  
Date                  
2017-03-14  0.158538  
2017-03-15  0.159186  
2017-03-16   0.15828  
2017-03-17  0.158114  
2017-03-20  0.157173  

Portfolio returns (head):
            Optimal_Portfolio_Return Equal_Weight_Return
Date                                                   
2017-03-14                -0.005398           -0.004688
2017-03-15                 0.011918            0.010684
2017-03-

In [5]:
import pandas as pd
import os

# Define the path to your CSV files
csv_folder = r"stage 2 result\multi-seed one time predict\10 iters"
output_csv = os.path.join(csv_folder, 'combined_weights.csv')

# Initialize an empty DataFrame to store combined results
combined_df = pd.DataFrame()

# Loop through each seed file
for seed in range(1, 2):  # Seed from 1 to 10
    file_path = os.path.join(csv_folder, f'ppo_stage2_weights_seed{seed}.csv')
    if os.path.exists(file_path):
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Add a column to indicate the seed number
        df['Seed'] = seed
        
        # Append to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"Warning: File {file_path} not found.")

# Save the combined DataFrame to a new CSV
combined_df.to_csv(output_csv, index=False)

print(f"Combined results saved successfully to {output_csv}.")

Combined results saved successfully to stage 2 result\multi-seed one time predict\10 iters\combined_weights.csv.


In [23]:
import pandas as pd
import numpy as np

# Load portfolio weights and daily returns
port_wts = pd.read_csv('port_wt_test.csv', parse_dates=['Date'], index_col='Date')
daily_returns = pd.read_csv('daily_returns_10ETFs.csv', parse_dates=['Date'], index_col='Date')

common_tickers = [col for col in port_wts.columns if col in daily_returns.columns]
daily_returns = daily_returns[common_tickers]

# Ensure daily_returns covers required dates
start_date, end_date = port_wts.index.min(), port_wts.index.max() + pd.Timedelta(days=21)
daily_returns = daily_returns.loc[start_date:end_date]

# Prepare final results DataFrame
cumulative_returns_df = pd.DataFrame(index=daily_returns.index)

# Loop through each run explicitly
for run in sorted(port_wts['Run'].unique()):
    print(f'process {run}')
    port_wts_run = port_wts[port_wts['Run'] == run].drop(columns=['Run'])

    # Initialize drifted weights
    drifted_weights = pd.DataFrame(index=daily_returns.index, columns=common_tickers)
    
    current_weights = port_wts_run.iloc[0].values

    returns_series = []

    for current_date in daily_returns.index:
        if current_date in port_wts_run.index:
            # On rebalance dates, explicitly assign new weights
            current_weights = port_wts_run.loc[current_date].values
        else:
            # Drift weights based on previous day's returns
            prev_returns = daily_returns.loc[current_date]
            current_weights = (current_weights * (1 + prev_returns.values))
            current_weights /= np.sum(current_weights)

        drifted_weights.loc[current_date] = current_weights
        
        # Use previous day's weights to calculate today's return
        if current_date == daily_returns.index[0]:
            daily_return = np.dot(current_weights, daily_returns.loc[current_date])
        else:
            prev_day_weights = drifted_weights.shift(1).loc[current_date]
            daily_return = np.dot(prev_day_weights, daily_returns.loc[current_date])

        returns_series.append(daily_return)

    cumulative_returns_df[f'Cumulative_Return_Run_{run}'] = (1 + pd.Series(returns_series, index=daily_returns.index)).cumprod() - 1

# Save final cumulative returns explicitly
cumulative_returns_df.to_csv('cumulative_returns_all_runs.csv')

print(cumulative_returns_df.head())


process 1
process 2
process 3
process 4
process 5
process 6
process 7
process 8
process 9
process 10
process 11
process 12
process 13
process 14
process 15
process 16
process 17
process 18
process 19
process 20
process 21
process 22
process 23
process 24
process 25
process 26
process 27
process 28
process 29
            Cumulative_Return_Run_1  Cumulative_Return_Run_2  \
Date                                                           
2017-03-14                -0.002668                -0.005907   
2017-03-15                 0.003042                 0.006372   
2017-03-16                 0.002931                 0.000685   
2017-03-17                -0.000860                -0.002089   
2017-03-20                -0.002660                -0.003773   

            Cumulative_Return_Run_3  Cumulative_Return_Run_4  \
Date                                                           
2017-03-14                -0.004572                -0.005295   
2017-03-15                 0.008736              

In [ ]:
import pandas as pd

# Load data explicitly
file_path = "E:/XAI/RL_with_SHAP/pythonProject1/SVERL_icml_2023/portfolio_DRL/results/consolidated_hierarchical_results.csv"
df = pd.read_csv(file_path)

# Identify stock columns (assuming columns BA to VZ inclusive)
start_col = 'BA'
end_col = 'VZ'
stock_columns = df.loc[:, start_col:end_col].columns.tolist()

# Perform grouping by 'Run' and 'Rebalance_Date' and calculate average
grouped_df = df.groupby(['Rebalance_Date'])[stock_columns].mean().reset_index()

# Export to a new CSV file
output_file = "averaged_stock_allocations.csv"
grouped_df.to_csv(output_file, index=False)

print(f"✅ Averaged data successfully saved to '{output_file}'")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# === Load your existing data ===
shap_metrics = pd.read_csv('shap_value_metrics_export.csv', parse_dates=['Start_Date', 'End_Date'])
daily_regimes = pd.read_csv('final_merged_data_with_forecast.csv', parse_dates=['Date'], index_col='Date')

# Extract relevant SHAP features for plotting
shap_features = [
    'mean_abs_shap_Mkt-RF',  # Market factor absolute SHAP
    'shap_std_Mkt-RF',        # Market factor SHAP dispersion
    'mean_abs_shap_SMB',      # Size factor absolute SHAP (as an example)
    'shap_std_SMB',           # Size factor SHAP dispersion (as an example)
]

# Aggregate daily SHAP features clearly
daily_shap = daily_regimes[shap_features].copy()
daily_shap.index = pd.to_datetime(daily_shap.index)

# Resample SHAP metrics to weekly averages for smoothness (optional)
weekly_shap = daily_shap.resample('W').mean()

# Get regime decisions (high-level PPO decisions)
weekly_regimes = daily_regimes['Cluster'].resample('W').last()

# === Plotting ===
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot Market factor mean absolute SHAP (indicator of market influence)
ax1.plot(
    weekly_shap.index,
    weekly_shap['mean_abs_shap_Mkt-RF'],
    label='Market Factor Abs SHAP (Influence Strength)',
    color='royalblue'
)
ax1.set_ylabel('Market Factor Abs SHAP', color='royalblue')
ax1.tick_params(axis='y', labelcolor='royalblue')

# Plot Market factor SHAP dispersion (indicator of volatility)
ax2 = ax1.twinx()
ax2.plot(
    weekly_shap.index,
    weekly_shap['shap_std_Mkt-RF'],
    label='Market Factor SHAP Std (Volatility)',
    color='gray'
)
ax2.set_ylabel('Market Factor SHAP Std', color='gray')
ax2.tick_params(axis='y', labelcolor='gray')

# Add vertical regime decision lines explicitly
for date, regime in weekly_regimes.items():
    if regime == 1:  # "Good" or Bull regime (customize based on your regime label meaning)
        ax1.axvline(x=date, color='green', linestyle='--', alpha=0.3, linewidth=2)
    elif regime == 0:  # "Bad" or Bear regime
        ax1.axvline(x=date, color='red', linestyle='--', alpha=0.3, linewidth=2)

# Explicitly annotate notable periods (e.g., 2020 sell-off)
ax1.axvspan('2020-02-01', '2020-04-30', color='red', alpha=0.1, label='2020 Market Selloff')
ax1.axvspan('2020-05-01', '2020-12-31', color='green', alpha=0.1, label='Post-stimulus Recovery')

# Titles and Legends
fig.suptitle('High-Level PPO Regime Decisions Interpreted by SHAP Signals', fontsize=16)
ax1.set_xlabel('Date')

# Construct a single legend combining lines and vertical regime lines
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper left')

# Tight layout for clarity
fig.tight_layout(rect=[0, 0.03, 1, 0.95])

# Save the figure
plt.savefig("regime_shap_interpretation.png", dpi=300)

# Display plot
plt.show()